# Comentarios

1. Se obtienen los rooms nulos o iguales a cero en primera instancia con expresiones regulares. No se obtienen primero con una media condicionada de la superficie cubierta y el lugar ya que es mas preciso obtenerlos con un dato directo del dataset

 **Pasos a seguir:**
1. Extraer precios y superficie con regex (terminado: refinar los patrones de busqueda para obtener mas datos)
2. Obtener el precio a partir del precio cuadrado y la superficie (terminado : no matcheo nada con las condiciones... falta poner igualmente la imputacion)
3. Obtener el precio de metro cuadrado (dolares y currency) mediante la otra variable y una conversion de moneda
4. Imputar las superficies por medio del precio y el precio por metro cuadrado (terminado: no matcheo nada con las condiciones...falta poner igualmente la imputacion)
5. Tomar placename+state name +cantidad de rooms sacar media condicionada para poner en surface
6. Imputar los rooms de los que pudimos extraer con regex
7. Una vez que imputamos la superficie y el precio, buscar el precio por m2

In [56]:
# Tratar de inputar los precios que faltan por expresiones regulares con la columna description y title
# Eliminar todas filas que todo es nulo (check)
# Checar el tema de las superficies
# Precios: unificar todos los precios a dolares
# Hacer la multiplicacion de price por la superficie
# Imputar superficies nulas por medias condicionadas por barrio o cantidad de rooms

# **Estandarizacion de nombres de mascaras, dataset y demas:**
- Al aplicarle una mascara a un dataset seria conveniente primero definir la mascara y en otra linea de codigo aplicarsela al dataset. Esto para que podamos ver bien que le estamos aplicando y que condiciones tiene esa mascara.
- Ojo con utilizar mascaras o alguna otra variable, dataframe y demas, definida mucho mas arriba. No vaya a ser que le cambiemos el nombre por x motivo y nos ollvidamos que mas abajo la usamos para otra cosa y se rompa todo
- Los nombres para las cosas que sean en ingles, separando las palabras con " _ " y en lo posible que se pueda intuir mediante el nombre a que se refiere
- Como practica totalmente arbitraria intentar que los nombres no tengan mas de 5 palabras. ej: " mask_rooms_zeros " , " mask_rooms_surf_zeros " y asi
- Si le llamamos " mask " a algo, que realmente lo sea. No que sea ya una mascara aplicada a un dataset. Eso es el dataset data filtrado por la mascara
- A estos nuevos dataset llamemosle igual que la mascara que le aplicamos solo que reemplazamos el nombre mask por data. ej:  " mask_rooms_zeros "=  " data_rooms_zeros ";                   " data_rooms_surf_zeros " = " data_rooms_surf_zeros "
- Cuando creemos un nuevo dataset, un array, una lista, una serie, mascara o lo que sea poner primero el nombre del tipo de objeto y despues lo que hace, para saber lo que es eso
- Cuando hacemos una mascara para ver si algo es nulo o cero ponemos solo " null ". Es para no alargar los nombres



### Criterios de analisis

- Primero decidimos con que instancias del DataSet vamos a realizar el modelo. En una primera instancia esta decision va a estar basada unicamente en el porcentaje de instancias que representa cada state_name.

# Importacion de librerias utilizadas

In [89]:
import pandas as pd
import numpy as np
import re

# Carga de la data de Properatti como un DataFrame de Pandas

In [90]:
data_location = "../Data/properatti.csv"

data = pd.read_csv(data_location, index_col=0)
data.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


# Renombro la columna lat-lon para evitar problemas

In [91]:
data.rename(columns={'lat-lon': 'lat_lon'}, inplace=True)

# Verificacion de instancias nulas

In [92]:
# Comprobamos la cantidad de filas donde tengan valor nulo en todas las columnas y las eliminamos  
data_clean_empty_row = data.dropna(axis = 0, how= 'all')
print(data.shape)
print(data_clean_empty_row.shape)
#En este caso no hay ninguna fila con todos los valores nulos

(121220, 25)
(121220, 25)


# Tratamiento de ceros en columnas

#### Dado que no hay ninguna variable que pueda tomar el valor cero, se van a reemplazar todos los valores ceros por NaN, para mejor tratamiento de los mismos

In [93]:
# Cantidad totales de ceros en el dataset
(data==0).sum()

operation                       0
property_type                   0
place_name                      0
place_with_parent_names         0
country_name                    0
state_name                      0
geonames_id                     0
lat_lon                         0
lat                             0
lon                             0
price                           1
currency                        0
price_aprox_local_currency      1
price_aprox_usd                 1
surface_total_in_m2           383
surface_covered_in_m2           2
price_usd_per_m2                0
price_per_m2                    0
floor                           0
rooms                           0
expenses                        0
properati_url                   0
description                     0
title                           0
image_thumbnail                 0
dtype: int64

In [94]:
# Cambio valores iguales a cero por NaN
data[data==0] = np.nan
(data==0).sum()

operation                     0
property_type                 0
place_name                    0
place_with_parent_names       0
country_name                  0
state_name                    0
geonames_id                   0
lat_lon                       0
lat                           0
lon                           0
price                         0
currency                      0
price_aprox_local_currency    0
price_aprox_usd               0
surface_total_in_m2           0
surface_covered_in_m2         0
price_usd_per_m2              0
price_per_m2                  0
floor                         0
rooms                         0
expenses                      0
properati_url                 0
description                   0
title                         0
image_thumbnail               0
dtype: int64

# Tratamiento de columnas con informacion no util o con informacion redundante

## Tratamiento de columnas *properati_url* y *image_thumbnail*

#### Vamos a desechar las columnas *properati_url* y *image_thumbnail* ya que no aportan informacion util

In [95]:
# desechamos las columnas properati_url y image_thumbnail de data
data = data.drop(['properati_url', 'image_thumbnail'], axis=1)
print("cantidad de columnas actuales=",data.shape[1])

cantidad de columnas actuales= 23


## Tratamiento de columnas *price* , *price_aprox_local_currency* y *price_aprox_usd* 

#### Vamos a quedarnos solo con la columna **price_aprox_usd** pudiendo desechar las columnas **price** y **price_aprox_local_currency** como asi tambien la columna **currency** ya que no aporta informacion adicional 
**Asumimos igual fiabilidad de los datos en las tres variables**

In [96]:
#Voy a verificar que no haya mas informacion en una columna que en las otras

#Hago tres mascaras para las variables de precios que contengan los nulos de las mismas
mask_price_null = data.price.isnull()
mask_price_aprox_local_currency_null = data.price_aprox_local_currency.isnull()
mask_price_aprox_usd_null = data.price_aprox_usd.isnull()

#Hago una interseccion de las tres mascaras
mask_prices_null = mask_price_null & mask_price_aprox_local_currency_null & mask_price_aprox_usd_null

#Si las tres mascaras tienen cada una la misma cantidad de elementos que la interseccion significa que los nulos se encuentran siempre en las mismas instancias
print("Cantidad de nulos en price= " , mask_price_null.sum())
print("cantidad de nulos en price_aprox_local_currency= " , mask_price_aprox_local_currency_null.sum())
print("Cantida de nulos en price_aprox_usd= " , mask_price_aprox_usd_null.sum())
print("Cantidad de nulos en la interseccion de las mascaras= " , mask_prices_null.sum())

#Devuelve un True si la cantidad de nulos es la misma en todas las mascaras por lo que la interseccion es perfecta
print(mask_price_null.sum()==mask_price_aprox_local_currency_null.sum()==mask_price_aprox_usd_null.sum()==mask_prices_null.sum())

Cantidad de nulos en price=  20411
cantidad de nulos en price_aprox_local_currency=  20411
Cantida de nulos en price_aprox_usd=  20411
Cantidad de nulos en la interseccion de las mascaras=  20411
True


In [97]:
#Desecho las columnas price, price_aprox_local_currency y currency
data = data.drop(['price', 'price_aprox_local_currency' , 'currency'], axis=1)
print("cantidad de columnas actuales= ",data.shape[1])

cantidad de columnas actuales=  20


## Tratamiento de columnas *price_usd_per_m2* y *price_per_m2*

#### Vamos a desechar la columna *price_per_m2* ya que es redundante con *price_usd_per_m2*
**Asumimos igual fiabilidad de datos en ambas variables**

In [98]:
#Voy a verificar que no haya mas informacion en una columna que en las otras

#Hago dos mascaras para las variables de precios por metro cuadrado que contengan los nulos de las mismas
mask_price_per_m2_null = data.price_per_m2.isnull()
mask_price_usd_per_m2_null = data.price_usd_per_m2.isnull()

#Hago una interseccion de las dos mascaras
mask_prices_per_m2_null = mask_price_per_m2_null & mask_price_usd_per_m2_null 

#Si las tres mascaras tienen cada una la misma cantidad de elementos que la interseccion significa que los nulos se encuentran siempre en las mismas instancias
print("Cantidad de nulos en price_per_m2= " , mask_price_per_m2_null.sum())
print("cantidad de nulos en price_usd_per_m2= " , mask_price_usd_per_m2_null.sum())
print("Cantidad de nulos en la interseccion de las mascaras= " , mask_prices_per_m2_null.sum())

#Devuelve un True si la cantidad de nulos es la misma en todas las mascaras por lo que la interseccion es perfecta
print(mask_price_per_m2_null.sum()==mask_price_usd_per_m2_null.sum()==mask_prices_per_m2_null.sum())

Cantidad de nulos en price_per_m2=  33562
cantidad de nulos en price_usd_per_m2=  52603
Cantidad de nulos en la interseccion de las mascaras=  28295
False


#### Como hay mas informacion en una columna que en la otra, veo en que instancias es nula la columna price_usd_per_m2 pero no lo es la columna price_per_m2 (solamente lo hago en esta ya que es la columna con la que nos vamos a quedar)

In [99]:
#Hago una mascara para ver cuando es nula la columna price_usd_per_m2 pero no lo es price_per_m2
mask_prices_per_m2 = data.price_usd_per_m2.isnull() & data.price_per_m2.notnull()
data_prices_per_m2 = data [mask_prices_per_m2]
data_prices_per_m2[["price_usd_per_m2" , "price_per_m2"]]

,price_usd_per_m2,price_per_m2
68,NaN,3263.888889
77,NaN,1764.705882
79,NaN,2809.523810
87,NaN,5147.058824
89,NaN,943.396226
...,...,...
121150,NaN,1826.923077
121151,NaN,1666.666667
121152,NaN,2257.575758
121155,NaN,1307.692308


In [100]:
#Realizo una imputacion en price_usd_per_m2 mediante price_per_m2

#Defino una variable que contenga el valor de conversion de pesos a dolar
pesos = 1/200

#Imputo en price_usd_per_m2 el valor de price_per_m2 multiplicado por el valor de la conversion de pesos a dolar
data.loc[data_prices_per_m2.index,"price_usd_per_m2"] = pesos *data.loc[data_prices_per_m2.index, "price_per_m2"]

In [101]:
#Desecho la columna price_per_m2
data = data.drop(['price_per_m2'], axis=1)
print("cantidad de columnas actuales= ",data.shape[1])

cantidad de columnas actuales=  19


## Tratamiento de columnas *lat-lon*, *lat* y *lon*

#### Vamos a desechar la columna *lat-lon* ya que es redundante con las columnas *lat* y *lon*
**Asumimos igual fiabilidad de datos en las tres variables**

In [102]:
#Voy a verificar que no haya mas informacion en una columna que en las otras

#Hago tres mascaras para las variables de latitud y longitud que contengan los nulos de las mismas
mask_lat_lon_null = data.lat_lon.isnull()
mask_lat_null = data.lat.isnull()
mask_lon_null = data.lon.isnull()

#Hago una interseccion de las tres mascaras
mask_inter_lat_lon_null = mask_lat_lon_null & mask_lat_null & mask_lon_null

#Si las tres mascaras tienen cada una la misma cantidad de elementos que la interseccion significa que los nulos se encuentran siempre en las mismas instancias
print("Cantidad de nulos en lat-lon= " , mask_lat_lon_null.sum())
print("cantidad de nulos en lat= " , mask_lat_null.sum())
print("Cantida de nulos en lon= " , mask_lon_null.sum())
print("Cantidad de nulos en la interseccion de las mascaras= " , mask_inter_lat_lon_null.sum())

#Devuelve un True si la cantidad de nulos es la misma en todas las mascaras por lo que la interseccion es perfecta
print(mask_lat_lon_null.sum()==mask_lat_null.sum()==mask_lon_null.sum()==mask_inter_lat_lon_null.sum())

Cantidad de nulos en lat-lon=  51550
cantidad de nulos en lat=  51550
Cantida de nulos en lon=  51550
Cantidad de nulos en la interseccion de las mascaras=  51550
True


In [103]:
#Desecho la columna lat_lon
data = data.drop(['lat_lon'], axis=1)
print("cantidad de columnas actuales= ",data.shape[1])

cantidad de columnas actuales=  18


# **REALIZADO DE CAMBIOS HASTA ACA, REVISAR MASCARAS Y DEMAS HACIA ABAJO**

# Limpieza de datos

# Descartado de instancias en funcion del state_name

In [ ]:
# Vemos todos los diferentes state_name que hay en el DataSet
data.state_name.unique()

In [ ]:
# Vemos la cantidad de instancias que tiene cada state_name
state_name_distribution = pd.value_counts(data.state_name)
state_name_distribution

In [ ]:
# Vemos los porcentajes de la distribucion de los datos por provincia
total_data = data.shape[0]
percentage_state_name = 100*state_name_distribution/total_data
percentage_state_name

### Tomamos la decision de desechar las instancias correspondientes a los *state_name* que representan < 1% del total de instancias.

In [ ]:
# Tomamos la decision de quedarnos con los datos que son > 1%, que corresponden a Buenos Aires Interior paara arriba
mask_1_percent = percentage_state_name > 1
series_1_percent = percentage_state_name [mask_1_percent]
print(f'La muestra representa el {round(series_1_percent.sum(), 2)}% del total de los datos')
round(series_1_percent, 2)

In [ ]:
# Con la mascara generada anteriormente para quedarnos con los state_names mayores a 1%, eliminamos las instancias que no corresponden a los state_names seleccionados
data = data[data.state_name.isin(series_1_percent.index)]
data

# Analis preliminar del DataSet

In [ ]:
# Usamos dtypes para saber que tipo de datos se almacenan en el dataset
data.dtypes

In [ ]:
#Vemos cuantos valores nulos tenemos por columna
is_null_result = data.isnull().sum()
print(is_null_result)

In [ ]:
data_cero = (data==0).sum()
data_cero

In [ ]:
#Vemos que porcentaje de nulos tiene cada columna
percentage_null = 100*is_null_result/data.shape[0]
percentage_null

## Correccion de instancias que tienen surface_total_in_m2 < surface_covered_in_m2

In [104]:
# Vemos en que casos el valor surface_total_in_m2 es menor que el valor surface_covered_in_m2
mask_surface = data.surface_total_in_m2 < data.surface_covered_in_m2
data_surface = data[mask_surface]

#En los casos en que surface_total_in_m2 sea menor que surface_covered_in_m2 imputamos los datos en el orden inverso
list_a = data_surface.surface_total_in_m2
list_b = data_surface.surface_covered_in_m2

data.loc[data_surface.index, "surface_total_in_m2"] = list_b
data.loc[data_surface.index, "surface_covered_in_m2"] = list_a

# Obtencion de datos con expresiones regulares de las columnas *description* y *title*

## Obtencion de rooms con expresiones regulares

##### *Se obtienen los rooms nulos o iguales a cero en primera instancia con expresiones regulares. No se obtienen primero con una media condicionada de la superficie cubierta y el lugar ya que es mas preciso obtenerlos con un dato directo del dataset*

### Obtencion de rooms con expresion regular en *description*

In [ ]:
# Aplicamos al DataSet una mascara booleana que contenga cero o nulo de rooms
mask_rooms_null = (data.rooms == 0) | (data.rooms.isnull())
data_rooms_null = data [mask_rooms_null]
series_rooms_null_description = data_rooms_null.description
series_rooms_null_description

In [ ]:
# Usamos expresiones regulares para sacar los rooms en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras amb indistindo de mayusculas o minusculas
#logica del patron: que tome un numero de un digito que solamente tenga detras uno o mas espacios y seguido de uno o mas espacios y las letras amb
pattern = '(?P<rooms>\d+)\s+(?i)amb' 
pattern_rooms_regex = re.compile(pattern)

In [ ]:
# Mostramos donde hay matches
description_rooms_result = series_rooms_null_description.apply(lambda x: pattern_rooms_regex.search(str(x)))
print(description_rooms_result)

In [ ]:
# Extraemos los datos que usaremos para la imputación (pongo que x sea cero cuando es none, ya que si me trae el valor None, no lo puedo pasar a float)
description_rooms_match = description_rooms_result.apply(lambda x: 0 if x is None else x.group('rooms'))
print(description_rooms_match)

print("Rooms matcheados distintos de cero=",(description_rooms_match!=0).sum()) #aca veo que cantidad pudo matchear
print(description_rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

In [ ]:
# Imputamos la serie rooms_match con su index en el DataSet 
data.loc[description_rooms_match.index, "rooms"] = description_rooms_match 
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

In [ ]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("int")
data[["rooms"]].dtypes
data.rooms.unique() #Aca quiero ver que valores unicos hay de rooms(acompanarlo con un grafico seria lo adecuado)

### Obtencion de rooms con expresion regular en *title*

In [ ]:
#Hacemos todo lo mismo que antes pero para obtener los rooms que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_rooms_zero = (data.rooms == 0)
data_rooms_zero = data [mask_rooms_zero]
series_rooms_zero_title = data_rooms_zero.title
series_rooms_zero_title

In [ ]:
title_rooms_result = series_rooms_zero_title.apply(lambda x: pattern_rooms_regex.search(str(x)))
print(title_rooms_result)

In [ ]:
title_rooms_match = title_rooms_result.apply(lambda x: 0 if x is None else x.group('rooms'))
print(title_rooms_match)

print("Rooms matcheados distintos de cero=",(title_rooms_match != 0).sum()) #aca veo que cantidad pudo matchear
print(title_rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

In [ ]:
# Imputamos la serie rooms_match_title con su index en el DataSet 
data.loc[title_rooms_match.index, "rooms"] = title_rooms_match 
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

In [ ]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("int")
data[["rooms"]].dtypes
data.rooms.unique() #Aca quiero ver que valores unicos hay de rooms(acompanarlo con un grafico seria lo adecuado)

## Obtencion de surface_total_in_m2 con expresiones regulares

##### *Se obtienen la superficie en la que ambas superficies sean nulas y el precio y el precio por metro cuadrado es nulo o cero. De esta manera sabemos que no podemos imputar una superficie con la otra y no podemos inferir la superficie mediante el precio y el precio por metro cuadrado*

### Obtencion de surface_total_in_m2 con expresion regular en *description*

In [ ]:
# Quiero ver que instancias son las que NO voy a poder imputar alguna superficie en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares
# Aplicamos al DataSet una mascara booleana que contenga cero o nulo de surface_covered_in_m2 y surface_total_in_m2 ademas de nulo en price y en price_per_m2 y price_usd_per_m2
surfaces_condition = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
prices_condition = ((data.price == 0) | (data.price.isnull())) & ((data.price_aprox_local_currency == 0) | (data.price_aprox_local_currency.isnull())) & ((data.price_aprox_usd == 0) | (data.price_aprox_usd.isnull()))
prices_per_m2_condition =  ((data.price_usd_per_m2 == 0) | (data.price_usd_per_m2.isnull())) & ((data.price_per_m2 == 0) | (data.price_per_m2.isnull()))

mask_surface_total_in_m2_null = surfaces_condition & (prices_condition | prices_per_m2_condition)
data_surface_total_in_m2_null = data [mask_surface_total_in_m2_null]

series_surface_total_in_m2_null_description = data_surface_total_in_m2_null.description
series_surface_total_in_m2_null_description

data_surface_total_in_m2_null.shape[0]

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en description y title buscando la superficie

In [ ]:
# Usamos expresiones regulares para sacar la superficie en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras m2 indistindo de mayusculas o minusculas
pattern = '(?P<surface_total_in_m2>\d+)\s+(?i)m2' 
pattern_surface_total_in_m2_regex = re.compile(pattern)

In [ ]:
# Mostramos donde hay matches
description_surface_total_in_m2_result = series_surface_total_in_m2_null_description.apply(lambda x: pattern_surface_total_in_m2_regex.search(str(x)))
print(description_rooms_result)

In [ ]:
description_surface_total_in_m2_match = description_surface_total_in_m2_result.apply(lambda x: 0 if x is None else x.group('surface_total_in_m2'))
print(description_surface_total_in_m2_match)

print("surfaces matcheados distintos de cero=",(description_surface_total_in_m2_match != 0).sum()) #aca veo que cantidad pudo matchear

In [ ]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[description_surface_total_in_m2_match.index, "surface_total_in_m2"] = description_surface_total_in_m2_match 
print(data[["property_type" , "surface_total_in_m2"]])
data[["surface_total_in_m2"]].dtypes

In [ ]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["surface_total_in_m2"] = data["surface_total_in_m2"].astype("float")
data[["surface_total_in_m2"]].dtypes

### Obtencion de surface_total_in_m2 con expresion regular en *title*

In [ ]:
#Hacemos todo lo mismo que antes pero para obtener las surface que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_surface_total_in_m2_zero = (data.surface_total_in_m2 == 0)
data_surface_total_in_m2_zero = data [mask_surface_total_in_m2_zero]
series_surface_total_in_m2_zero_title = data_surface_total_in_m2_zero.title
series_surface_total_in_m2_zero_title

In [ ]:
title_surface_total_in_m2_result = series_surface_total_in_m2_zero_title.apply(lambda x: pattern_surface_total_in_m2_regex.search(str(x)))
print(title_surface_total_in_m2_result)

In [ ]:
title_surface_total_in_m2_match = title_surface_total_in_m2_result.apply(lambda x: 0 if x is None else x.group('surface_total_in_m2'))
print(title_surface_total_in_m2_match)

print("surfaces matcheados distintos de cero=",(title_surface_total_in_m2_match != 0).sum()) #aca veo que cantidad pudo matchear

In [ ]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[title_surface_total_in_m2_match.index, "surface_total_in_m2"] = title_surface_total_in_m2_match 
print(data[["property_type" , "surface_total_in_m2"]])
data[["surface_total_in_m2"]].dtypes

In [ ]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["surface_total_in_m2"] = data["surface_total_in_m2"].astype("float")
data[["surface_total_in_m2"]].dtypes

## Obtencion de price mediante expresiones regulares

##### *Quiero obtener el price para las instancias en las que no tenemos price y price_aprox_local_currency y price_aprox_usd y tampoco tenemos surface_total_in_m2 y surface_total_in_m2 o no tenemos price_usd_per_m2 y price_per_m2 *

### Obtencion de price con expresion regular en *description*

In [ ]:
# Quiero ver que instancias son las que NO voy a poder imputar algun price en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares

surfaces_condition = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
prices_condition = ((data.price == 0) | (data.price.isnull())) & ((data.price_aprox_local_currency == 0) | (data.price_aprox_local_currency.isnull())) & ((data.price_aprox_usd == 0) | (data.price_aprox_usd.isnull()))
prices_per_m2_condition =  ((data.price_usd_per_m2 == 0) | (data.price_usd_per_m2.isnull())) & ((data.price_per_m2 == 0) | (data.price_per_m2.isnull()))


mask_price_null = prices_condition & (prices_per_m2_condition | surfaces_condition)
data_price_null = data [mask_price_null]
series_price_null_description = data_price_null.description
series_price_null_description

data_price_null.shape[0]

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en description y title buscando el price

In [ ]:
# Usamos expresiones regulares para sacar la superficie en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras m2 indistindo de mayusculas o minusculas
pattern = '(?P<price>\d+)\s+(?i)usd' 
pattern_price_regex = re.compile(pattern)

In [ ]:
# Mostramos donde hay matches
description_price_result = series_price_null_description.apply(lambda x: pattern_price_regex.search(str(x)))
print(description_price_result)

In [ ]:
description_price_match = description_price_result.apply(lambda x: 0 if x is None else x.group('price'))
print(description_price_match)

print("price matcheados distintos de cero=",(description_price_match != 0).sum()) #aca veo que cantidad pudo matchear

In [ ]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[description_price_match.index, "price"] = description_price_match 
print(data[["property_type" , "price"]])
data[["price"]].dtypes

In [ ]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["price"] = data["price"].astype("float")
data[["price"]].dtypes

### Obtencion de price con expresion regular en *title*

In [ ]:
#Hacemos todo lo mismo que antes pero para obtener las surface que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_price_zero = (data.price == 0)
data_price_zero = data [mask_price_zero]
series_price_zero_title = data_price_zero.title
series_price_zero_title

In [ ]:
title_price_result = series_price_zero_title.apply(lambda x: pattern_price_regex.search(str(x)))
print(title_price_result)

In [ ]:
title_price_match = title_price_result.apply(lambda x: 0 if x is None else x.group('price'))
print(title_price_match)

print("price matcheados distintos de cero=",(title_price_match != 0).sum()) #aca veo que cantidad pudo matchear

In [ ]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[title_price_match.index, "price"] = title_price_match 
print(data[["property_type" , "price"]])
data[["price"]].dtypes

In [ ]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["price"] = data["price"].astype("float")
data[["price"]].dtypes

# Obtencion de datos mediante imputacion directa

## Obtencion de place_name mediante imputacion directa

In [ ]:
#Analizamos los nulos en la columna place_name con una mascara booleana
serie_place_name = data.place_name
mask_place_name_null = serie_place_name.isnull()
data_place_name_null = data[mask_place_name_null]
data_place_name_null

#### *Como todos los nulos en la columna place_name corresponden al municipio de Tigre, los imputamos directamente con este municipo*

In [ ]:
# Como todos los nulos en la columna place_name corresponden al municipio de Tigre, los imputamos directamente con este municipio
data.loc[data_place_name_null.index, "place_name"] = 'Tigre'

## Obtencion de surface_total_in_m2 y surface_covered_in_m2 mediante imputacion directa

### Datos de Superficie total o superficie cubierta que quedaran nulos o ceros por ahora

In [ ]:
# Realizamos una mascara para ver las instancias de superficie tanto cubierta como total de los cuales no tenemos para imputar un valor con el otro

mask_surfaces_null = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
data_surfaces_null = data[mask_surfaces_null]

print (data_surfaces_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#En este DataFrame hay tanto superficies nulas como superficies iguales a cero

### Datos de superficie total o superficie que imputo uno con el otro

In [ ]:
# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie total pero que la cubierta no es nula ni cero, por lo tanto se la podemos imputar
mask_surface_total_in_m2_null = (data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()
data_surface_total_in_m2_null = data[mask_surface_total_in_m2_null]

# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie cubierta pero que la total no es nula ni cero, por lo tanto se la podemos imputar
mask_surface_covered_in_m2_null = ((data.surface_total_in_m2 != 0) & (data.surface_total_in_m2.notnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
data_surface_covered_in_m2_null = data[mask_surface_covered_in_m2_null]


print (data_surface_total_in_m2_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])
print (data_surface_covered_in_m2_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])


In [ ]:
# Aca imputamos los valores nulos o ceros de superficie total con los valores de superficie cubierta, utilizando la mascara mask_surface_total_in_m2_zero
data.loc[data_surface_total_in_m2_null.index, "surface_total_in_m2"] = data.loc[data_surface_total_in_m2_null.index, "surface_covered_in_m2"]

# Aca imputamos los valores nulos o ceros de superficie cubierta con los valores de superficie total, utilizando la mascara mask_surface_covered_in_m2_zero
data.loc[data_surface_covered_in_m2_null.index, "surface_covered_in_m2"] = data.loc[data_surface_covered_in_m2_null.index, "surface_total_in_m2"]

### Obtención de las superficias nulas en funcion del precio y del precio por metro cuadrado

##### Vemos instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [ ]:
#Aca veo las instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [ ]:
# Se hacen tres mascaras booleanas: mask_surfaces_null = instancias sin datos de superficie / mask_prices_notnull = instancias con algun dato de precio / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_null = ((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))
condition_prices_notnull = ((data.price != 0) & (data.price.notnull())) | ((data.price_aprox_local_currency != 0) & (data.price_aprox_local_currency.notnull())) | ((data.price_aprox_usd != 0) & (data.price_aprox_usd.notnull()))
condition_prices_per_m2_notnull = ((data.price_usd_per_m2!=0) & (data.price_usd_per_m2.notnull())) | ((data.price_per_m2!=0) & (data.price_per_m2.notnull()))

mask_surfaces_imput = condition_surfaces_null & condition_prices_notnull & condition_prices_per_m2_notnull
data_surfaces_imput = data[mask_surfaces_imput]

data_surfaces_imput[["price" , "price_aprox_local_currency" , "price_aprox_usd" , "price_usd_per_m2" , "price_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

## Obtencion de price mediante imputacion directa en funcion de prices_per_m2 y surface

In [ ]:
# Se hacen tres mascaras booleanas: mask_surfaces_nonull = instancias con datos de superficie no nulos / mask_prices_null = instancias sin datos de precios / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_notnull = ((data.surface_total_in_m2 != 0) & (data.surface_total_in_m2.notnull())) | ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))
condition_prices_null = ((data.price == 0) | (data.price.isnull())) & ((data.price_aprox_local_currency == 0) | (data.price_aprox_local_currency.isnull())) & ((data.price_aprox_usd == 0) | (data.price_aprox_usd.isnull()))
condition_prices_per_m2_notnull = ((data.price_usd_per_m2!=0) & (data.price_usd_per_m2.notnull())) | ((data.price_per_m2!=0) & (data.price_per_m2.notnull()))

mask_prices_imput = condition_surfaces_null & condition_prices_notnull & condition_prices_per_m2_notnull
data_prices_imput = data[mask_prices_imput]

data_prices_imput[["price" , "price_aprox_local_currency" , "price_aprox_usd" , "price_usd_per_m2" , "price_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

## Obtencion de price_per_m2 mediante imputacion directa en funcion de surface y price

In [ ]:
# Se hacen tres mascaras booleanas: mask_surfaces_null = instancias sin datos de superficie / mask_prices_notnull = instancias con algun dato de precio / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_notnull = ((data.surface_total_in_m2 != 0) & (data.surface_total_in_m2.notnull())) | ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))
condition_prices_notnull = ((data.price != 0) & (data.price.notnull())) | ((data.price_aprox_local_currency != 0) & (data.price_aprox_local_currency.notnull())) | ((data.price_aprox_usd != 0) & (data.price_aprox_usd.notnull()))
condition_prices_per_m2_null = ((data.price_usd_per_m2 ==0) | (data.price_usd_per_m2.isnull())) & ((data.price_per_m2 ==0) | (data.price_per_m2.isnull()))

mask_prices_per_m2_imput = condition_surfaces_notnull & condition_prices_notnull & condition_prices_per_m2_null
data_prices_per_m2_imput = data[mask_prices_per_m2_imput]

data_prices_per_m2_imput[["price" , "price_aprox_local_currency" , "price_aprox_usd" , "price_usd_per_m2" , "price_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

In [ ]:
data_prices_per_m2_imput.notnull().sum()

## En revision: Buscar con una expresion regular imputar la superficie con los datos de 'description' y 'title'

## Analisis de columna rooms

In [ ]:
#Aca veo, de los rooms que son iguales a cero, cuales tienen la superficie cubierta distinta de cero o nulo para imputarla despues
data_rooms_surface = data[(data.rooms == 0) & ((data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0) | (data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0))]
data_rooms_surface[["rooms" , "surface_covered_in_m2"]]

# *Analisis columnas de precios*

Hacemos una mascara para nulos dolar y nulos local m2

In [ ]:
data_price_null = data[((data.price_usd_per_m2.isnull()) | (data.price_usd_per_m2 == 0)) & ((data.price_per_m2.isnull()) | (data.price_per_m2== 0))]
data_price_null[['price', 'price_usd_per_m2', 'price_per_m2', 'surface_total_in_m2']]
# Estas instancias son las que tendremos que imputar por superficio y precio, si no tiene alguno se tendran que explorar nuevas opciones
print(data_price_null.isnull().sum())
print(data_price_null.shape)

In [ ]:
data_price_surface_notnull = data_price_null[((data_price_null.price.notnull()) & (data_price_null.price != 0)) & ((data_price_null.surface_total_in_m2.notnull()) & (data_price_null.surface_total_in_m2 != 0))]
data_price_surface_notnull

In [ ]:
data.price[20]

In [ ]:
data.surface_total_in_m2[20]

# **VENIMOS CODEANDO HASTA ACA, NO DAR BOLA A LO QUE SIGUE**

In [ ]:

# Queremos, para los que tengan ambas superficies nulas o cero se las imputamos por una media condicionada por el valor de property_type, place_name, state_name y rooms

In [ ]:
moneda = data[data.currency=="UYU"]
moneda.description[107390]

In [ ]:
data.country_name.unique()

In [ ]:
#Aca hago una maquinita manual para obtener el numero de instancia de los valores raros de rooms
marcaradedatos =  data[data.rooms == "0002"]
marcaradedatos.description[43292]

In [ ]:
#Aca empiezo a tomar una muestra para ver que pasa

In [ ]:
data.description[121134     ]

In [ ]:
data.description[1601]

In [ ]:
data.description[116017]

In [ ]:
data.description[76675	]

In [ ]:
data.description[7689]

## Analisis de columna price

### Intentaremos imputar las instancias de precios nulas o iguales a cero con expresiones regulares en la columna descripcion

In [ ]:
data[data.surface_total_in_m2.isnull()]

In [ ]:
data[surfaces_condition]

In [ ]:
mask_prices_surfaces_prices_m2_null = (surfaces_condition & prices_condition) | prices_per_m2_condition
data[mask_prices_surfaces_prices_m2_null]

In [ ]:
prices_condition